<a href="https://colab.research.google.com/github/fatimasadiq/NLP_Module_Project_hatespeechdetection/blob/main/NLP_Project_askfm_task2_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing Libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import string
string.punctuation
import re
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
#text lower
df["q_a"] = df["q_a"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['q_a']= df['q_a'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['q_a']= df['q_a'].apply(lambda x:remove_numbers(x))
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,idk im not gay,0
2,this person is fake so beware isabella,1
3,oof lmao how is she fake whos this lmao shit f...,1
4,if your loved one left n cut all ties w you sh...,0


In [ ]:
# LR and TFIDF after removing punctuations and numbers

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report ,accuracy_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['q_a'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train q_a length: ',len(train_x))
print('test q_a length: ',len(valid_x))

train q_a length:  7998
test q_a length:  2000


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['q_a'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['q_a'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['q_a'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['q_a'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.9035
LR F1score, WordLevel TF-IDF:  0.9259904863050262
LR Accuracy, N-Gram Vectors:  0.888
LR F1score, N-Gram Vectors:  0.9278175287356322
LR Accuracy, CharLevel Vectors:  0.9085
LR F1score, N-Gram Vectors:  0.9286855688718708


In [ ]:
#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["q_a"] = df["q_a"].apply(lambda text: remove_stopwords(text))
df.head()

,q_a,label
0,tell guy gay seem super straight,0
1,Idk I'm gay,0
2,This person 100% fake beware Isabella!,1
3,Oof lmao fake? Who's lmao shit fuck speak like...,1
4,If loved one left n cut ties w u wait bc I sti...,0


In [ ]:
#Accuracy and F1score after stopwords removal

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report ,accuracy_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['q_a'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train q_a length: ',len(train_x))
print('test q_a length: ',len(valid_x))

train q_a length:  7998
test q_a length:  2000


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['q_a'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['q_a'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['q_a'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['q_a'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.9005
LR F1score, WordLevel TF-IDF:  0.9245423438164099
LR Accuracy, N-Gram Vectors:  0.8755
LR F1score, N-Gram Vectors:  0.9298484642832798
LR Accuracy, CharLevel Vectors:  0.8985
LR F1score, N-Gram Vectors:  0.9225879334568631


In [ ]:
#Accuracy and F1score after Stemming

In [ ]:
#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
#stemming
def stem_words(text):
    return " ".join([porter_stemmer.stem(word) for word in text.split()])

df["q_a"] = df["q_a"].apply(lambda text: stem_words(text))
df.head()

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,idk i'm not gay,0
2,thi person is 100% fake so bewar isabella!,1
3,oof lmao how is she fake? who' thi lmao shit f...,1
4,If your love one left n cut all tie w you shou...,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report ,accuracy_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['q_a'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train q_a length: ',len(train_x))
print('test q_a length: ',len(valid_x))

train q_a length:  7998
test q_a length:  2000


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['q_a'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['q_a'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['q_a'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['q_a'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.905
LR F1score, WordLevel TF-IDF:  0.9265437158469946
LR Accuracy, N-Gram Vectors:  0.886
LR F1score, N-Gram Vectors:  0.925861325401989
LR Accuracy, CharLevel Vectors:  0.904
LR F1score, N-Gram Vectors:  0.9269889941883827


In [ ]:
#Accuracy and F1score after applying all the preprocessing steps

In [ ]:
#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
#by applying all the preprocessing steps at once

In [ ]:
#text lower
df["q_a"] = df["q_a"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['q_a']= df['q_a'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['q_a']= df['q_a'].apply(lambda x:remove_numbers(x))
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,idk im not gay,0
2,this person is fake so beware isabella,1
3,oof lmao how is she fake whos this lmao shit f...,1
4,if your loved one left n cut all ties w you sh...,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["q_a"] = df["q_a"].apply(lambda text: remove_stopwords(text))
df.head()

,q_a,label
0,tell guy gay seem super straight,0
1,idk im gay,0
2,person fake beware isabella,1
3,oof lmao fake whos lmao shit fuck speak like c...,1
4,loved one left n cut ties w u wait bc still fe...,0


In [ ]:
dfb =df[['label' ,'q_a']]
dfb.head(5)

,label,q_a
0,0,tell guy gay seem super straight
1,0,idk im gay
2,1,person fake beware isabella
3,1,oof lmao fake whos lmao shit fuck speak like c...
4,0,loved one left n cut ties w u wait bc still fe...


In [ ]:
dfb[1:2000].to_csv("askfmberttest.csv" , index=False)

In [ ]:
dfb[2001:10000].to_csv("askfmberttrain.csv" , index=False)

In [ ]:
#stemming
def stem_words(text):
    return " ".join([porter_stemmer.stem(word) for word in text.split()])

df["q_a"] = df["q_a"].apply(lambda text: stem_words(text))
df.head()

,q_a,label
0,tell guy gay seem super straight,0
1,idk im gay,0
2,person fake bewar isabella,1
3,oof lmao fake who lmao shit fuck speak like cu...,1
4,love one left n cut tie w u wait bc still feel,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['q_a'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train q_a length: ',len(train_x))
print('test q_a length: ',len(valid_x))

train q_a length:  7998
test q_a length:  2000


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['q_a'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

AttributeError: ignored

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['q_a'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['q_a'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['q_a'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR Accuracy, N-Gram Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR Accuracy, CharLevel Vectors: ", accuracy)
print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.8935
LR F1score, WordLevel TF-IDF:  0.9206473822113763
LR Accuracy, N-Gram Vectors:  0.8785
LR F1score, N-Gram Vectors:  0.9290024979101051
LR Accuracy, CharLevel Vectors:  0.9005
LR F1score, N-Gram Vectors:  0.9216497087783048


In [ ]:
#Accuracy and F1 score after sentiment analysis

In [ ]:
#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
#text lower
df["q_a"] = df["q_a"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['q_a']= df['q_a'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['q_a']= df['q_a'].apply(lambda x:remove_numbers(x))
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,idk im not gay,0
2,this person is fake so beware isabella,1
3,oof lmao how is she fake whos this lmao shit f...,1
4,if your loved one left n cut all ties w you sh...,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["q_a"] = df["q_a"].apply(lambda text: remove_stopwords(text))
df.head()

,q_a,label
0,tell guy gay seem super straight,0
1,idk im gay,0
2,person fake beware isabella,1
3,oof lmao fake whos lmao shit fuck speak like c...,1
4,loved one left n cut ties w u wait bc still fe...,0


In [ ]:
#Sentiment Analysis
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
# Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
# polarity_scores method of SentimentIntensityAnalyzer
# oject gives a sentiment dictionary.
# which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    #print("Overall sentiment dictionary is : ", sentiment_dict)
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    #print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return 1 #positive
    elif sentiment_dict['compound'] <= - 0.05 :
        return -1 #negative
    else :
        return 0 #neutral
    

In [ ]:
# Driver code
if __name__ == "__main__" :
    print("Text Selected for VADER Sentimental Analysis :")
    sentence1 = ('''ANN is like our brain; millions and billions of cells — called neurons, which processes information in the form of electric signals. Similarly, in ANN, the network structure has an input layer, a hidden layer, and the output layer. It is also called Multi-Layer Perceptron as it has multiple layers. The hidden layer is known as a “distillation layer” that distils some critical patterns from the data/information and passes it onto the next layer. It then makes the network quicker and more productive by distinguishing the data from the data sources, leaving out the excess data.''')
    print(sentence1)
#sentiment_scores(sentence1)


Text Selected for VADER Sentimental Analysis :
ANN is like our brain; millions and billions of cells — called neurons, which processes information in the form of electric signals. Similarly, in ANN, the network structure has an input layer, a hidden layer, and the output layer. It is also called Multi-Layer Perceptron as it has multiple layers. The hidden layer is known as a “distillation layer” that distils some critical patterns from the data/information and passes it onto the next layer. It then makes the network quicker and more productive by distinguishing the data from the data sources, leaving out the excess data.


In [ ]:
df["sentiment"] = df["q_a"].apply(lambda text: sentiment_scores(text))
df.head(10)

,q_a,label,sentiment
0,tell guy gay seem super straight,0,1
1,idk im gay,0,-1
2,person fake beware isabella,1,-1
3,oof lmao fake whos lmao shit fuck speak like c...,1,1
4,loved one left n cut ties w u wait bc still fe...,0,1
5,bitch,1,-1
6,oof damn,0,-1
7,help,0,1
8,farts fam,0,0
9,lmfao wtf nosence u talking u know behave,1,-1


In [ ]:
df1=df[['label' ,'sentiment']]
df1.head()

,label,sentiment
0,0,1
1,0,-1
2,1,-1
3,1,1
4,0,1


In [ ]:
df1['sentiment']=df1['sentiment'].astype(str)
df1['sentiment']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0        1
1       -1
2       -1
3        1
4        1
        ..
9993     0
9994    -1
9995    -1
9996     1
9997     0
Name: sentiment, Length: 9998, dtype: object

In [ ]:
df1['label']=df1['label'].astype(str)
df1['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       0
1       0
2       1
3       1
4       0
       ..
9993    0
9994    0
9995    0
9996    0
9997    0
Name: label, Length: 9998, dtype: object

In [ ]:
df1[1:2000].to_csv("berttestsenti.csv" , index=False)

In [ ]:
df1[2001:10000].to_csv("berttrainsenti.csv" , index=False)

In [ ]:
df1.to_csv("askfm-sentiments.csv" , index=False)

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df1['label'], df1['sentiment'], test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train q_a length: ',len(train_x))
print('test q_a length: ',len(valid_x))

train q_a length:  7998
test q_a length:  2000


In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, lowercase=False)
tfidf_vect.fit(df1['sentiment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
#tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
#tfidf_vect_ngram.fit(df1['sentiment'])
#xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
#xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
#tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
#tfidf_vect_ngram_chars.fit(df1['sentiment'])
#xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
#xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy, f1score

In [ ]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR Accuracy, WordLevel TF-IDF: ", accuracy)
print("LR F1score, WordLevel TF-IDF: ", f1score)

# Linear Classifier on Ngram Level TF IDF Vectors
#accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print("LR Accuracy, N-Gram Vectors: ", accuracy)
#print("LR F1score, N-Gram Vectors: ", f1score)

# Linear Classifier on Character Level TF IDF Vectors
#accuracy,f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
#print("LR Accuracy, CharLevel Vectors: ", accuracy)
#print("LR F1score, N-Gram Vectors: ", f1score)

LR Accuracy, WordLevel TF-IDF:  0.43
LR F1score, WordLevel TF-IDF:  0.5366480722676094


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/sentiments.zip

Archive:  gdrive/My Drive/sentiments.zip
  inflating: askfm-berttrain.csv     
  inflating: __MACOSX/._askfm-berttrain.csv  
  inflating: askfm-bert-test.csv     
  inflating: __MACOSX/._askfm-bert-test.csv  


In [ ]:
!unzip gdrive/My\ Drive/wiki-news-300d-1M.vec

Archive:  gdrive/My Drive/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
import csv
df2=pd.read_csv('/content/wiki-news-300d-1M.vec' ,quoting=csv.QUOTE_NONE, error_bad_lines=False , delimiter=";")
#df2.head(5)

b'Skipping line 42: expected 1 fields, saw 2\n'


In [ ]:
#Data Loading 
df=pd.read_csv('/content/AskFm_datasets.csv' , error_bad_lines=False, delimiter=",") #error_bad_lines=False usecols=["label","q_a"]
#df['label']=df['label'].astype('object')
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,Idk I'm not gay,0
2,This person is 100% fake so beware Isabella!,1
3,Oof lmao how is she fake? Who's this lmao shit...,1
4,If your loved one left n cut all ties w you sh...,0


In [ ]:
#text lower
df["q_a"] = df["q_a"].str.lower()

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
df['q_a']= df['q_a'].apply(lambda x:remove_punctuation(x))

#removing numbers 
def remove_numbers(text):
  output=''.join(c for c in text if not c.isdigit())
  return output
#applying the function
#text=this person 100 fake beware isabella
df['q_a']= df['q_a'].apply(lambda x:remove_numbers(x))
df.head(5)

,q_a,label
0,how to tell if a guy is gay if they seem super...,0
1,idk im not gay,0
2,this person is fake so beware isabella,1
3,oof lmao how is she fake whos this lmao shit f...,1
4,if your loved one left n cut all ties w you sh...,0


In [ ]:
#removing stop words
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in stopwords])

df["q_a"] = df["q_a"].apply(lambda text: remove_stopwords(text))
df.head()

,q_a,label
0,tell guy gay seem super straight,0
1,idk im gay,0
2,person fake beware isabella,1
3,oof lmao fake whos lmao shit fuck speak like c...,1
4,loved one left n cut ties w u wait bc still fe...,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score ,classification_report

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['q_a'], df['label'] , test_size=0.2, random_state=42)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
print('train q_a length: ',len(train_x))
print('test q_a length: ',len(valid_x))

train q_a length:  7998
test q_a length:  2000


In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'.')
count_vect.fit(df['q_a'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['q_a'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['q_a'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['q_a'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    #print(metrics.classification_report(valid_y, predictions, target_names=['q_a', 'label']))
    #f1score=metrics.f1_score(predictions,valid_y , average='weighted')
    #accuracy=metrics.accuracy_score(predictions, valid_y)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.f1_score(predictions,valid_y , average='weighted')

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/wiki-news-300d-1M.vec' , encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df['q_a'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf 
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=tf.optimizers.Adam(), loss='binary_crossentropy')
    
    return model
classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

250/250 [==============================] - 7s 24ms/step - loss: 0.3056
CNN, Word Embeddings 0.9319092122830441


In [ ]:
import tensorflow as tf 
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=tf.optimizers.Adam(), loss='binary_crossentropy')
    
    return model
classifier = create_cnn()
f1score = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  f1score)

250/250 [==============================] - 8s 28ms/step - loss: 0.3118
CNN, Word Embeddings 0.9384288747346072
